will load the library first

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import pandas  as pd
from io import StringIO
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import metrics


Data Exploration

In [3]:
data = pd.read_csv ('Women Clothing E-Commerce Review.csv')   

info

In [4]:
data.info

<bound method DataFrame.info of       id  Clothing_ID  Age                                              Title  \
0      2         1077   60                            Some major design flaws   
1      3         1049   50                                   My favorite buy!   
2     16         1065   34  You need to be at least average height, or taller   
3     17          853   41                       Looks great with white pants   
4     18         1120   32                                Super cute and cozy   
..   ...          ...  ...                                                ...   
321  416          872   35                                  I love this shirt   
322  417         1083   35                                             Flimsy   
323  418          831   40                                        Super comfy   
324  419         1080   32                          Pretty pattern, weird fit   
325  420         1077   68                                   Easy and stylish

missing values

In [5]:
data.isna().sum()

id                         0
Clothing_ID                0
Age                        0
Title                      0
ReviewText                 0
Rating                     0
Recommended                0
Positive Feedback Count    0
Division Name              0
product                    0
Class_Name                 0
dtype: int64

we need only

In [6]:
col = ['ReviewText', 'product']
data = data[col]
data.columns =['ReviewText', 'product']

catogrize the product

In [7]:
data['product_id'] = data['product'].factorize()[0]
product_id_data = data[['product', 'product_id']].drop_duplicates().sort_values('product_id')
product_to_id = dict(product_id_data.values)
id_to_product = dict(product_id_data[['product_id', 'product']].values)

C:\ff\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


see our new data

In [8]:
data.head()

,ReviewText,product,product_id
0,I had such high hopes for this dress and reall...,Dresses,0
1,"I love, love, love this jumpsuit. it's fun, fl...",Bottoms,1
2,Material and color is nice. the leg opening i...,Bottoms,1
3,Took a chance on this blouse and so glad i did...,Tops,2
4,"A flattering, super cozy coat. will work well...",Jackets,3


vectorizr

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(data.ReviewText).toarray()


set the label

In [10]:
labels = data.product_id

split the data

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data['ReviewText'], data['product'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

train the model

In [15]:
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),#Linear Support Vector Classification.
    MultinomialNB(),#Naive Bayes classifier for multinomial models
    LogisticRegression(random_state=0),
]
CV = 10
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

NameError: name 'cross_val_score' is not defined

see the accuracy

In [157]:
cv_df.groupby('model_name').accuracy.mean()

model_name
LinearSVC                 0.760985
LogisticRegression        0.632292
MultinomialNB             0.558428
RandomForestClassifier    0.546117
Name: accuracy, dtype: float64

try the lll

In [160]:
model = LinearSVC()
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, data.index, test_size=0.33, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

see matrix

In [163]:
print(metrics.classification_report(y_test, y_pred, 
                                    target_names=data['product'].unique()))

              precision    recall  f1-score   support

     Dresses       0.92      0.86      0.89        14
     Bottoms       0.71      0.45      0.56        22
        Tops       0.68      0.93      0.78        56
     Jackets       1.00      0.44      0.62         9
    Intimate       0.00      0.00      0.00         7

    accuracy                           0.72       108
   macro avg       0.66      0.54      0.57       108
weighted avg       0.70      0.72      0.69       108

